### LangChain and OpenAI LLM Quick Demo

This demo showcases how to use LangChain to build useful large language model applications. LangChain is a framework that allows us to interact with Large Language Models (LLMs) like OpenAI GPTs.

First, install the required modules.

Always best to upgrade since `langchain` is rapidliy developing.


In [1]:
! pip install langchain --upgrade
! pip install openai --upgrade

Then, import the packages to use.

Please create your [OpenAI](https://openai.com/) key before proceeding. 

In [2]:
import os
import textwrap
from langchain.llms import OpenAI

query = input("OpenAI key: ")
os.environ["OPENAI_API_KEY"] = query

#### Demo 1: A Simple Question and Answering Example

The simplest example is a question and answering application. To build the demo, execute the following: 

- Load OpenAI Large Language Model (`llm`). Temperature is a measure of uncertainty or randomness of answers. Higher temperature means more randomness. In other words, temp = 0 means the model will always give the same answer. Temp = 1 means the model will give answers with more variance. Let's set it to 0.5.
- Provide a text query
- Request the Large Language Model to process the query
- Print the result

In [3]:
llm = OpenAI(temperature=0.5)
text = "What are the 10 best places to visit?"
print(llm(text))



1. Paris, France
2. London, England
3. Rome, Italy
4. New York City, USA
5. Barcelona, Spain
6. Tokyo, Japan
7. Amsterdam, Netherlands
8. Prague, Czech Republic
9. Istanbul, Turkey
10. Hong Kong, China


#### Demo 2: Using a Prompt Template

A meaningful interaction with a LLM requires a good prompt to elicit a good answer. In this example a prompt template is provided to elicit a good answer from the LLM. 

In [4]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["to_do_what"],
    template="What are the 10 best places to {to_do_what}?",
)
formatted_prompt = prompt.format(to_do_what="eat")
print(f"Formatted prompt: {formatted_prompt}")

Formatted prompt: What are the 10 best places to eat?


#### Demo 3: A Simple Chain

LLM is not very useful unless we can interact with it using prompt. In the same way, a prompt is not valuable unless we can use it to interact with a LLM. A chain puts them together. Below is a simple chain.

Chain: Prompt &rarr; LLM 

Then, the chain can now be queried.

`chain.run(<to_do_what>`

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
formatted_prompt = prompt.format(to_do_what="...")
query = input(formatted_prompt)
formatted_prompt = prompt.format(to_do_what=query)
print(f"Human: {formatted_prompt}")
results = chain.run(query).split('\n')
print(f"AI: ")
for result in results:
    print(result)

Human: What are the 10 best places to to eat street food?
AI: 


1. La Boqueria Market, Barcelona, Spain
2. Khao San Road, Bangkok, Thailand
3. Chiang Mai Night Bazaar, Thailand
4. Tsukiji Fish Market, Tokyo, Japan
5. La Rambla, Barcelona, Spain
6. La Boca, Buenos Aires, Argentina
7. Old Delhi, India
8. St. Lawrence Market, Toronto, Canada
9. La Paz, Bolivia
10. Georgetown, Penang, Malaysia


#### Demo 4: Enabling Long-Term Memory 

A conversation has context. Sometimes, the context has a reference to a discussion long time ago. Without the ability to remember something said a while back, the conversation can easily become frustrating. Thus, memory is important. In this example, a memory is incorporated in the chain.

Below is a conversation with context.

In [11]:
from langchain import OpenAI, ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(temperature=0.5)
# we use a memory to store the conversation history
memory = ConversationBufferMemory() 
conversation = ConversationChain(
        llm=llm, 
        verbose=True, 
        memory=memory
        )

conversation.predict(input="Hi! I want to talk to you. What is your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi! I want to talk to you. What is your name?
AI:

> Finished chain.


" Hi there! My name is AI-Bot. It's nice to meet you. What would you like to talk about?"

##### Let's Build a Conversation Chain

The human can ask questions all about a topic. Assume that there is context in the conversation. To end the conversation, the human can say "bye". 

In [12]:
while True:
    input_prompt = "Human: "
    query = input(input_prompt)

    if query.lower() == "bye":
        print("AI: Bye!")
        # print the conversation history
        print(memory.chat_memory)
        break

    text = f"{input_prompt}: {query}"
    print(textwrap.fill(text, width=80))
    text = f"AI: {conversation.predict(input=query)}"
    print(textwrap.fill(text, width=80))

Human: : I want to learn about AI. Where do I start?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi! I want to talk to you. What is your name?
AI:  Hi there! My name is AI-Bot. It's nice to meet you. What would you like to talk about?
Human: I want to learn about AI. Where do I start?
AI:

> Finished chain.
AI:  Great question! You can start by learning about the basics of AI and its
history. AI has been around for decades, but has become increasingly popular in
recent years. There are many resources available online that can help you get
started.
Human: : What is the cheapest and quickest way to master AI?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly co

#### Demo 5: Using Agent and Tools to Determine what Actions to Take to Solve a Problem

Sometimes, a problem can not be solved in one step. Additional tools may also needed in order to solve the problem. In this demo, the LLM uses an agent to determine what actions to take and what tools to use.

The focus is on math problems. `llm-math` tool is used. More info about [tools](https://langchain.readthedocs.io/en/latest/modules/agents/tools.html) and [agents](https://langchain.readthedocs.io/en/latest/modules/agents/agents.html) are available in the LangChain documentation.

In [8]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [10]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0.0)

# Next, let's load some tools to use. 
# Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, 
# and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Now let's test it out!

query = input("Human: ")
text = f"Human: {query}"
agent.run(query)



> Entering new AgentExecutor chain...
 I need to find a prime number
Action: Calculator
Action Input: List of prime numbers less than 50
Observation: Answer: 328
Thought: I need to find the smallest prime number
Action: Calculator
Action Input: Sort the list of prime numbers in ascending order

ValueError: 'VariableNode' object has no attribute 'sort'. Please try again with a valid numerical expression

### Demo 6: Evaluation

Evaluation of chains is difficult to perform because of: 1) Lack of data and 2) Lack of metrics.

- Lack of data

Unlike machine learning problems, there is not enough data to evaluate chains. As such, the evaluation is performed on specific examples only. For example, Q&A on documents with known answers. LangChain provides a small number of datasets for evaluation.

- Lack of metrics

Unlike machine learning problems where there are more or less precise answers (e.g. accuracy on classification, mAP on detection, etc.), there are no precise answers in language. As such, the evaluation is more complicated. How do you evaluate the quality of a conversation or essay?

In [13]:
# Comment this out if you are NOT using tracing
import os

from langchain.evaluation.loading import load_dataset

os.environ["LANGCHAIN_HANDLER"] = "langchain"
dataset = load_dataset("question-answering-state-of-the-union")

ModuleNotFoundError: No module named 'datasets'